This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/IMDB).

## IMDB Sample

The following example demonstrates how to use Wallaroo with chained models.  In this example, we will be using information from the IMDB (Internet Movie DataBase) with a sentiment model to detect whether a given review is positive or negative.  Imagine using this to automatically scan Tweets regarding your product and finding either customers who need help or have nice things to say about your product.

Note that this example is considered a "toy" model - only the first 100 words in the review were tokenized, and the embedding is very small.

The following example is based on the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/), and sample data can be downloaded from the [aclIMDB dataset](http://s3.amazonaws.com/text-datasets/aclImdb.zip ).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from IPython.display import display

In [2]:
import pandas as pd
import os
os.environ["ARROW_ENABLED"]="False"
if "ARROW_ENABLED" in os.environ:
    arrowEnabled = os.environ["ARROW_ENABLED"]
else:
    arrowEnabled = False
    
# used to display dataframe information without truncating
pd.set_option('display.max_colwidth', None)

In [3]:
# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR SUFFIX"


# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                 auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                 auth_type="sso")

wl = wallaroo.Client()

To test this model, we will perform the following:

* Create a workspace for our models.
* Upload two models:
  * `embedder`: Takes pre-tokenized text documents (model input: 100 integers/datum; output 800 numbers/datum) and creates an embedding from them.
  * `sentiment`:  The second model classifies the resulting embeddings from 0 to 1, which 0 being an unfavorable review, 1 being a favorable review.
* Create a pipeline that will take incoming data and pass it to the embedder, which will pass the output to the sentiment model, and then export the final result.
* To test it, we will use information that has already been tokenized and submit it to our pipeline and gauge the results.

First we'll create a workspace for our environment, and call it `imdbworkspace`.  We'll also set up our pipeline so it's ready for our models.

In [4]:
workspace_name = 'imdbworkspace'
pipeline_name = 'imdbpipeline'

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

imdb_pipeline = get_pipeline(pipeline_name)
imdb_pipeline

name,imdbpipeline
created,2023-02-10 23:10:18.355050+00:00
last_updated,2023-02-10 23:10:18.355050+00:00
deployed,(none)
tags,
versions,ca986da6-21cc-47f3-a29a-77359e5ac8fb
steps,


Just to make sure, let's list our current workspace.  If everything is going right, it will show us we're in the `imdb-workspace`.

In [7]:
wl.get_current_workspace()

{'name': 'imdbworkspace', 'id': 32, 'archived': False, 'created_by': '39bf22e5-bb55-40ce-b1ab-36eed7a29150', 'created_at': '2023-02-10T23:10:18.178492+00:00', 'models': [], 'pipelines': [{'name': 'imdbpipeline', 'create_time': datetime.datetime(2023, 2, 10, 23, 10, 18, 355050, tzinfo=tzutc()), 'definition': '[]'}]}

Now we'll upload our two models:

* `embedder.onnx`: This will be used to embed the tokenized documents for evaluation.
* `sentiment_model.onnx`: This will be used to analyze the review and determine if it is a positive or negative review.  The closer to 0, the more likely it is a negative review, while the closer to 1 the more likely it is to be a positive review.

In [8]:
embedder = wl.upload_model('embedder-o', './embedder.onnx').configure()
smodel = wl.upload_model('smodel-o', './sentiment_model.onnx').configure(runtime="onnx", tensor_fields=["flatten_1"])

With our models uploaded, now we'll create our pipeline that will contain two steps:

* First, it runs the data through the embedder.
* Second, it applies it to our sentiment model.

In [9]:
# now make a pipeline
imdb_pipeline.add_model_step(embedder)
imdb_pipeline.add_model_step(smodel)

name,imdbpipeline
created,2023-02-10 23:10:18.355050+00:00
last_updated,2023-02-10 23:10:18.355050+00:00
deployed,(none)
tags,
versions,ca986da6-21cc-47f3-a29a-77359e5ac8fb
steps,


Now that we have our pipeline set up with the steps, we can deploy the pipeline.

In [10]:
imdb_pipeline.deploy()

Waiting for deployment - this will take up to 45s ............. ok


name,imdbpipeline
created,2023-02-10 23:10:18.355050+00:00
last_updated,2023-02-10 23:10:19.026267+00:00
deployed,True
tags,
versions,"8d9f40f2-4905-40b0-a8bc-3c95876af442, ca986da6-21cc-47f3-a29a-77359e5ac8fb"
steps,embedder-o


We'll check the pipeline status to verify it's deployed and the models are ready.

In [11]:
imdb_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.2.84',
   'name': 'engine-945766797-tvhh4',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'imdbpipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'smodel-o',
      'version': '0efe6f5b-1496-434d-9be4-c0caecb636a1',
      'sha': '3473ea8700fbf1a1a8bfb112554a0dde8aab36758030dcde94a9357a83fd5650',
      'status': 'Running'},
     {'name': 'embedder-o',
      'version': '50ab44f1-f821-4ec7-878d-6472d4fd46b3',
      'sha': 'd083fd87fa84451904f71ab8b9adfa88580beb92ca77c046800f79780a20b7e4',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.1.38',
   'name': 'engine-lb-74b4969486-q4pk9',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

To test this out, we'll start with a single piece of information from our data directory.

In [12]:
if arrowEnabled==True:
    # using Dataframe JSON
    results = results = imdb_pipeline.infer_from_file('./data/singleton.df.json')
    display(results)
else:
    # pre-arrow Wallaroo JSON
    results = imdb_pipeline.infer_from_file('./data/singleton.json')
    display(results[0].data())

,time,in.tensor,out.dense_1,check_failures
0,2023-02-10 23:10:32.978,"[1607.0, 2635.0, 5749.0, 199.0, 49.0, 351.0, 16.0, 2919.0, 159.0, 5092.0, 2457.0, 8.0, 11.0, 1252.0, 507.0, 42.0, 287.0, 316.0, 15.0, 65.0, 136.0, 2.0, 133.0, 16.0, 4311.0, 131.0, 286.0, 153.0, 5.0, 2826.0, 175.0, 54.0, 548.0, 48.0, 1.0, 17.0, 9.0, 183.0, 1.0, 111.0, 15.0, 1.0, 17.0, 284.0, 982.0, 18.0, 28.0, 211.0, 1.0, 1382.0, 8.0, 146.0, 1.0, 19.0, 12.0, 9.0, 13.0, 21.0, 1898.0, 122.0, 14.0, 70.0, 14.0, 9.0, 97.0, 25.0, 74.0, 1.0, 189.0, 12.0, 9.0, 6.0, 31.0, 3.0, 244.0, 2497.0, 3659.0, 2.0, 665.0, 2497.0, 63.0, 180.0, 1.0, 17.0, 6.0, 287.0, 3.0, 646.0, 44.0, 15.0, 161.0, 50.0, 71.0, 438.0, 351.0, 31.0, 5749.0, 2.0, 0.0, 0.0]",[0.37142318],0


Since that works, let's load up all 50 rows and do a full inference on each of them.  Note that Jupyter Hub has a size limitation, so for production systems the outputs should be piped out to a different output.

In [13]:
if arrowEnabled==True:
    # Arrow enabled dataframe
    results = imdb_pipeline.infer_from_file('./data/test_data.df.json')
    # just display the first row for space
    display(results.loc[0,:])
else:
    # pre-arrow Wallaroo JSON
    results = imdb_pipeline.infer_from_file('./data/test_data.json')
    display(results[0].data())

time                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                2023-02-10 23:10:33.036000
in.tensor         [1607.0, 2635.0, 5749.0, 199.0, 49.0, 351.0, 16.0, 2919.0, 159.0, 5092.0, 2457.0, 8.0, 11.0, 1252.0, 507.0, 42.0, 287.0, 316.0, 15.0, 65.0, 136.0, 2.0, 133.0, 16.0, 4311.0, 131.0, 286.0, 153.0, 5.0, 2826.0, 175.0, 54.0, 548.0, 48.0, 1.0, 17.0, 9.0, 183.0, 1.0, 111.0, 15.0, 1.0, 17.0, 284.0, 982.0, 18.0, 28.0, 211.0, 1.0, 1382

## Undeploy

With our pipeline's work done, we'll undeploy it and give our Kubernetes environment back its resources.

In [14]:
imdb_pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ..................................... ok


name,imdbpipeline
created,2023-02-10 23:10:18.355050+00:00
last_updated,2023-02-10 23:10:19.026267+00:00
deployed,False
tags,
versions,"8d9f40f2-4905-40b0-a8bc-3c95876af442, ca986da6-21cc-47f3-a29a-77359e5ac8fb"
steps,embedder-o


And there is our example. Please feel free to contact us at Wallaroo for if you have any questions.